In [1]:
from selenium import webdriver
driver = webdriver.Firefox()

In [2]:
import pandas as pd
import time

In [3]:
links = ['https://abledata.acl.gov/m/products?field_product_status=70320&search_api_views_fulltext=%20Visually%20impaired']
for i in range(1,49):
    links.append('https://abledata.acl.gov/m/products?field_product_status=70320&search_api_views_fulltext=%20Visually%20impaired&page='+str(i))

df = pd.DataFrame(columns=['Sno.','PName','Desc','Image','Plink'])

In [4]:
driver.get(links[0])

In [33]:
for ml in links[15:]:
    driver.get(ml)
    pnl = driver.find_elements_by_xpath('//h2[@class="field-content"]/a')
    pnames = [x.text for x in pnl]
    pls = driver.find_elements_by_xpath('//h2[@class="field-content"]/a')
    plinks = [x.get_attribute('href') for x in pls]
    piml = driver.find_elements_by_xpath('//div[@class="views-field views-field-field-product-image solrproduct_img"]/span/a/img')
    pims = [x.get_attribute('src') for x in piml]
    pdesc = []
    for pl in plinks:
        driver.get(pl)
        pdesc.append(driver.find_element_by_xpath('//div[@class="field field-name-body field-type-text-with-summary field-label-hidden"]/div/div').text)
    for i in range(len(pnames)):
        l = len(df)
        df.loc[l,'Sno.'] = l+1
        df.loc[l,'PName'] = pnames[i]
        df.loc[l,'Desc'] = pdesc[i]
        df.loc[l,'Image'] = pims[i]
        df.loc[l,'Plink'] = plinks[i]
    print(len(df))
    

170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
497


,Sno.,PName,Desc,Image,Plink
0,1,Intro To The Piano For The Visually Impaired &...,Intro to the Piano for the Visually Impaired a...,https://abledata.acl.gov/sites/default/files/s...,https://abledata.acl.gov/product/intro-piano-v...
1,2,Co-Robotic Navigation Aid for the Visually Imp...,The Co-Robotic Navigation Aid for the Visually...,https://abledata.acl.gov/sites/default/files/s...,https://abledata.acl.gov/product/co-robotic-na...
2,3,Visually Impaired Assistant (Via),-------- PROTOTYPE --------- PURPOSE: To desig...,https://abledata.acl.gov/sites/default/files/s...,https://abledata.acl.gov/product/visually-impa...
3,4,Getting To Know You: A Social Skills and Abili...,Getting To Know You is a social skills/awarene...,https://abledata.acl.gov/sites/default/files/s...,https://abledata.acl.gov/product/getting-know-...
4,5,Navigational Aids For The Visually Impaired (N...,---- PROTOTYPE --------- PURPOSE: To create a ...,https://abledata.acl.gov/sites/default/files/s...,https://abledata.acl.gov/product/navigational-...
5,6,Plan.b,---- PROTOTYPE --------- PURPOSE: To design a ...,https://abledata.acl.gov/sites/default/files/s...,https://abledata.acl.gov/product/planb
6,7,DIY Talking Shampoo/Conditioner Bottle for the...,Talking Shampoo/Conditioner Bottle for the Eld...,https://abledata.acl.gov/sites/default/files/s...,https://abledata.acl.gov/product/diy-talking-s...
7,8,Sentire Walking Cane,---- PROTOTYPE --------- PURPOSE: To design a ...,https://abledata.acl.gov/sites/default/files/s...,https://abledata.acl.gov/product/sentire-walki...
8,9,Anple,---- PROTOTYPE --------- PURPOSE: To design a ...,https://abledata.acl.gov/sites/default/files/s...,https://abledata.acl.gov/product/anple
9,10,Braille Keyboard Stickers for the Blind and Vi...,Braille Keyboard Stickers for the Blind and Vi...,https://abledata.acl.gov/sites/default/files/s...,https://abledata.acl.gov/product/braille-keybo...


In [4]:
import nltk.tokenize as nt
import nltk
from nltk.stem.wordnet import WordNetLemmatizer


In [5]:
def extract_NN(sent):
    grammar = r"""
    NBAR:
        # Nouns and Adjectives, terminated with Nouns
        {<NN.*>*<NN.*>}

    NP:
        {<NBAR>}
        # Above, connected with in/of/etc...
        {<NBAR><IN><NBAR>}
    """
    chunker = nltk.RegexpParser(grammar)
    ne = set()
    chunk = chunker.parse(nltk.pos_tag(nltk.word_tokenize(sent)))
    for tree in chunk.subtrees(filter=lambda t: t.label() == 'NP'):
        ne.add(' '.join([child[0] for child in tree.leaves()]))
    return ne

def getKeywords(text):
    keyw = extract_NN(text)
    lem = WordNetLemmatizer()
    keyw = list(set([lem.lemmatize(word) for word in keyw])) 
    return keyw
        

In [6]:
df = pd.DataFrame(columns=['Sno.','PName','Desc','Keywords','Date'])

In [8]:
for ml in links[42:]:
    driver.get(ml)
    pnl = driver.find_elements_by_xpath('//h2[@class="field-content"]/a')
    pnames = [x.text for x in pnl]
    pls = driver.find_elements_by_xpath('//h2[@class="field-content"]/a')
    plinks = [x.get_attribute('href') for x in pls]
    pds = driver.find_elements_by_xpath('//span[@class="date-display-single"]')
    pdate = [x.text for x in pds]
    pkeywords = [] 
    pdesc = []
    for pl in plinks:
        driver.get(pl)
        try:
            desc = ''+driver.find_elements_by_xpath('//div[@class="field field-name-body field-type-text-with-summary field-label-hidden"]/div/div/p')[0].text
            try:
                desc += driver.find_elements_by_xpath('//div[@class="field field-name-body field-type-text-with-summary field-label-hidden"]/div/div/p')[1].text
                desc += driver.find_elements_by_xpath('//div[@class="field field-name-body field-type-text-with-summary field-label-hidden"]/div/div/p')[2].text
            except:
                pass
        except:
            desc = ''+driver.find_element_by_xpath('//div[@class="field field-name-body field-type-text-with-summary field-label-hidden"]/div/div').text
        pdesc.append(desc)
        pkeywords.append(getKeywords(desc))
    for i in range(len(pnames)):
        l = len(df)
        df.loc[l,'Sno.'] = l+1
        df.loc[l,'PName'] = pnames[i]
        df.loc[l,'Desc'] = pdesc[i]
        df.loc[l,'Keywords'] = pkeywords[i]
        df.loc[l,'Date'] = pdate[i]
        df.loc[l,'Plink'] = plinks[i]
        print(len(df))



421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487


In [9]:
df.to_excel('Data1.xls',index=False)

In [11]:
for i in range(len(df)):
    for j in ['http','VIA','TITLE','PROTOTYPE','REF','user','users']:
        if(j in df.loc[i,'Keywords']):
            df.loc[i,'Keywords'].remove(j)
        if(j.lower() in df.loc[i,'Keywords']):
            df.loc[i,'Keywords'].remove(j.lower())

In [14]:
for i in range(len(df)):
    df.loc[i,'Date'] = df.loc[i,'Date'].split('/')[2]

IndexError: list index out of range

In [17]:
df1=df

In [18]:
lsa = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
lca = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

In [21]:
import retinasdk
import re
import time
liteClient = retinasdk.LiteClient("23e7a440-f56b-11e9-8f72-af685da1b20e")
for i in range(len(df1)):
    r1 = ' '.join(df1.loc[i,'Keywords'])
    r1 = re.sub(r"[^a-zA-Z]+", ' ', r1)
    df1.loc[i,'Keywords'] = liteClient.getKeywords(r1)
    time.sleep(10)
    print(i)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [24]:
df1.to_excel('FinalDataProducts.xls',index=False)

In [49]:
wordcount = {}
for i in range(len(df1)):
    for j in df1.loc[i,'Keywords']:
        if(j in wordcount.keys()):
            wordcount[j]+=1
        else:
            wordcount[j]=1
        

In [50]:
wordcount = sorted(wordcount.items(), key = lambda kv:(kv[1], kv[0]))

In [51]:
wordcount.reverse()

In [52]:
dfk = pd.DataFrame(columns=['Keywords','WordCount'])
for i in range(len(wordcount)):
    dfk.loc[i,'Keywords'] = wordcount[i][0]
    dfk.loc[i,'WordCount'] = wordcount[i][1]
    

In [53]:
dfk

,Keywords,WordCount
0,device,84
1,vision,64
2,color,53
3,braille,52
4,inch,50
5,button,48
6,compatibility,36
7,app,35
8,warranty,33
9,dimensions,33


In [54]:
dfk.to_excel('KeywordCountProducts.xls',index=False)

In [116]:
# Selenium code for scrapping through cortical.io
driver.get('http://www.cortical.io/extract-keywords.html')

for i in range(len(df)):
    text = df.loc[i,'Desc']
    tip = driver.find_element_by_id('text-input')
    tip.clear()
    tip.send_keys(text)
    while True:
        try:
            time.sleep(2)
            driver.find_element_by_id('submit-text').click()
            break
        except:
            continue
    time.sleep(5)
    kls = driver.find_elements_by_xpath('//div[@id="standard-output"]/span')
    df.loc[i,'Keywords'] = [x.text for x in kls]

In [31]:
#Code to create alpha list
i = 'A'
lsa = [i]
for j in range(25):
    i = chr(ord(i)+1)
    lsa.append(i)
print('[')
for i in lsa:
    print('\''+i+'\''+',',end='')
    


In [121]:
df.to_excel('NewDataToCheck.xls',index=False)

In [30]:
def stringtolist(li):
    return [x.strip("'") for x in li.strip('][').split(', ')]
stringtolist("['1', '2', '3']")

1
2
3


In [62]:
liteClient.getSimilarTerms("")

['github',
 'api',
 'plugins',
 'gpl',
 'open-source',
 'developers',
 'linux',
 'firefox',
 'bsd',
 'functionality',
 'plugin',
 'mozilla',
 'gnu',
 'compiler',
 'proprietary',
 'implementations',
 'ibm',
 'org',
 'gnu general public license',
 'mac os']

In [7]:
import difflib
seq = difflib.get_close_matches('education','')
d = seq.ratio()*100
d

12.5

In [ ]:
df = pd.read_csv()